File for all CV Methods combined so that it runs in a single pipeline

1) To install all required packages for this pipeline, use the `installPackages.sh` script.

2) Imports

In [ ]:
#using workaround with rclone
#from google.colab.patches import cv2_imshow #can not use on linux
#from google.colab import drive #can not use on linux

import csv, json, os, math, random, re, sys, statistics
import numpy as np
import pandas as pd
import cv2
import torch, torchvision
import detectron2

from natsort import natsorted, ns
from colorthief import ColorThief
from fer import FER
from PIL import Image
import matplotlib.pyplot as plt
import skimage.io as io #scikit-image

from torch.autograd import Variable as V
from torchvision import transforms as trn
from torch.nn import functional as F
import torchvision.models as models

from detectron2.utils.logger import setup_logger
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer, ColorMode
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
#setup logger for detectron2
setup_logger()

In [ ]:
class AgeGender:
  def __init__(self, one):
    # Defined the model files
    self.FACE_PROTO = "./models/opencv_face_detector.pbtxt"
    self.FACE_MODEL = "./models/opencv_face_detector_uint8.pb"
    self.AGE_PROTO = "./models/age_deploy.prototxt"
    self.AGE_MODEL = "./models/age_net.caffemodel"
    self.GENDER_PROTO = "./models/gender_deploy.prototxt"
    self.GENDER_MODEL = "./models/gender_net.caffemodel"

    # Load network
    self.FACE_NET = cv2.dnn.readNet(self.FACE_MODEL, self.FACE_PROTO)
    self.AGE_NET = cv2.dnn.readNet(self.AGE_MODEL, self.AGE_PROTO)
    self.GENDER_NET = cv2.dnn.readNet(self.GENDER_MODEL, self.GENDER_PROTO)

    self.MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
    self.AGE_LIST = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]
    self.GENDER_LIST = ["Male", "Female"]

    self.box_padding = 20

    self.frameName = []
    self.person = []
    self.boxFace = []
    self.gen = []
    self.gender_conf = []
    self.ages = []
    self.age_conf = []

  def get_face_box (self, net, frame, conf_threshold = 0.5):
    frame_copy = frame.copy()
    frame_height = frame_copy.shape[0]
    frame_width = frame_copy.shape[1]
    blob = cv2.dnn.blobFromImage(frame_copy, 1.0, (300, 300), [104, 117, 123], True, False)

    net.setInput(blob)
    detections = net.forward()
    boxes = []

    for i in range(detections.shape[2]):
      confidence = detections[0, 0, i, 2]

      if confidence > conf_threshold:
        x1 = int(detections[0, 0, i, 3] * frame_width)
        y1 = int(detections[0, 0, i, 4] * frame_height)
        x2 = int(detections[0, 0, i, 5] * frame_width)
        y2 = int(detections[0, 0, i, 6] * frame_height)
        boxes.append([x1, y1, x2, y2])
        cv2.rectangle(frame_copy, (x1, y1), (x2, y2), (0, 255, 0), int(round(frame_height / 150)), 8)

    return frame_copy, boxes

  def age_gender_detector (self, input_path, filename):
    image = cv2.imread(input_path)
    resized_image = cv2.resize(image, (640, 480))

    frame = resized_image.copy()
    frame_face, boxes = self.get_face_box(self.FACE_NET, frame)

    count = 0
    for box in boxes:
      self.frameName.append(filename)
      self.person.append(count)
      self.boxFace.append(box)
      face = frame[max(0, box[1] - self.box_padding):min(box[3] + self.box_padding, frame.shape[0] - 1), \
        max(0, box[0] - self.box_padding):min(box[2] + self.box_padding, frame.shape[1] - 1)]

      blob = cv2.dnn.blobFromImage(face, 1.0, (227, 227), self.MODEL_MEAN_VALUES, swapRB = False)
      self.GENDER_NET.setInput(blob)
      gender_predictions = self.GENDER_NET.forward()
      gender = self.GENDER_LIST[gender_predictions[0].argmax()]
      self.gen.append(gender)
      self.gender_conf.append(gender_predictions[0].max())

      self.AGE_NET.setInput(blob)
      age_predictions = self.AGE_NET.forward()
      age = self.AGE_LIST[age_predictions[0].argmax()]
      self.ages.append(age)
      self.age_conf.append(age_predictions[0].max())

      label = "{},{}".format(gender, age)
      cv2.putText(frame_face, label, (box[0], box[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2, cv2.LINE_AA)
      cv2.putText(frame_face, str(count), (box[0] + 2, box[1] + 15), cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),1,cv2.LINE_AA, )

      count += 1

#write to file extra
    if len(boxes) > 0:
      cv2.resize(frame_face, (640, 480))
      cv2.imwrite("/content/gdrive/MyDrive/digital_humanities/Ergebnisse/Agender/Siedler/" + filename, frame_face)

    return frame_face

  def x(self):
    # we rerun the pipeline for each game, so assign path with __init__
    directory = os.fsencode("assetpath with current game")
    count = 0
    for file in os.listdir(directory):
      count += 1
      if count == 100: #use other number here
        break
      
      filename = os.fsdecode(file)
      age_gender_detector("/content/gdrive/MyDrive/digital_humanities/Walkthroughts/Siedler_Frames/" + filename, filename)

  def saveData(self):
    df = pd.DataFrame(list(zip(self.frameName, self.person, self.boxFace, self.gen, self.gender_conf, self.ages, self.age_conf)))
    df.columns = ['Name', 'person', 'box', 'gender', 'gender_conf', 'age', 'age_conf']
    #change output
    df.to_csv("/content/gdrive/MyDrive/digital_humanities/Ergebnisse/Agender/siedler_agender.csv")

In [2]:
games = 1 #list of the game images in assets/*
directory = os.fsencode("./assets/")
dir1 = os.listdir("./assets/")

print(dir1)

#hopefully there is no DS_Store XD
if len(games) != 8:
  raise Exception("assets does not contain ")


while games:
  #init ageGender
  #process ageGender
  #save ageGender

  #merge color methods in one class

  #emotion
  #location
  #object detection

  #tipp:
  #do not load the assets multiple times!
  #pass the data with each loop iteration
  break

['NFSMW', 'Die_Siedler3', 'Elden_Ring', 'Super_Mario_World', 'Starcraft2', 'Diablo2', 'Half_Life2', 'Little_Nightmares']


TypeError: object of type 'int' has no len()